https://discuss.streamlit.io/t/maps-with-altair-and-geopandas/2953
Geopandas não trabalha com Streamlit
Aparentemente existem soluções para isso

Talvez esta solução dê certo. https://www.programcreek.com/python/?code=xoolive%2Ftraffic%2Ftraffic-master%2Ftraffic%2Fcore%2Fmixins.py

In [1]:
import pandas as pd
import altair as alt
import geopandas as gpd
import json
import numpy as np
import urllib
import voila
%config Completer.use_jedi = False
#alt.themes.enable('opaque')
#ThemeRegistry.enable('opaque')

In [2]:
df_matriz = pd.read_excel('../../../matriz_londrina.xls'
                          , sheet_name='matriz_pretratada'
                          ,dtype={'cod_mun':np.int64}
                         )
df_matriz.head()

,cod_mun,rm,municipio,TERRITORIO_arranjo,TERRITORIO_rm_londrina,TERRITORIO_sedu2017,TERRITORIO_metronorte,TERRITORIO_nre,TERRITORIO_nrs,ESCALA_area_km2,...,URB_RUR_densidade_hab_ha2020,URB_RUR_hierarquia,MOB_indice_integracao,MOB_distancia_reta_km,MOB_distancia_reta_km.1,MOB_tempo_deslocamento,SOCIOE_pib_percapita_2018,SOCIOE_idh,SOCIOE_vulnerabilidade,SOCIOE_Firjan
0,4113809,RML,Lupionópolis,Não,Sim,Não,Não,Sim,Sim,121.066,...,0,1 Centro Local,0.115964,80,103,0,24880.58,0.710,0.253,0.5213
1,4105102,RML,Centenário do Sul,Não,Sim,Não,Não,Sim,Sim,371.834,...,0,1 Centro Local,0.210375,67,98,0,20943.44,0.668,0.349,0.5014
2,4103404,RML,Cafeara,Não,Não,Não,Não,Sim,Sim,185.800,...,0,1 Centro Local,0.215490,81,109,0,23156.44,0.693,0.276,0.5350
3,4109203,RML,Guaraci,Não,Sim,Não,Não,Sim,Sim,211.680,...,0,1 Centro Local,0.190262,65,85,0,19423.76,0.698,0.238,0.5333
4,4116000,RML,Miraselva,Não,Sim,Não,Não,Sim,Sim,90.294,...,0,1 Centro Local,0.212627,49,72,0,21369.53,0.748,0.195,0.5613


In [ ]:
gdf_municipios = gpd.read_file('../../../../../maps/shp/territorio/BR_Municipios_2019.shp')

In [ ]:
gdf_municipios = gdf_municipios.loc[gdf_municipios['CD_MUN'].isin([
    '4113809', '4105102', '4103404', '4109203', '4116000', '4120002', '4108007',
    '4100806', '4120507', '4111902', '4120333', '4102802', '4126504', '4113700',
    '4112702', '4109807', '4103701', '4119657', '4122701', '4101507', '4122404',
    '4101903', '4126678', '4126405', '4121307', '4128401'])]
gdf_municipios.reset_index(drop=True, inplace=True)
gdf_municipios['CD_MUN'] = gdf_municipios['CD_MUN'].astype(np.int64)

In [ ]:
gdf_municipios

In [ ]:
gdf_municipios = gdf_municipios.merge(df_matriz, left_on='CD_MUN', right_on='cod_mun')

In [ ]:
gdf_municipios.columns

In [ ]:
map_rm = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.Color('TERRITORIO_rm_londrina:N', legend=None),tooltip='NM_MUN:N').properties(width=500,height=200, title='Região Metropolitana')
map_arranjo = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.Color('TERRITORIO_arranjo:N', legend=alt.Legend(title='Pertence')), tooltip='NM_MUN:N').properties(width=500,height=200, title='Arranjo Populacional')
map_metronorte = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.Color('TERRITORIO_metronorte:N', legend=None),tooltip='NM_MUN:N').properties(width=500,height=200, title='Metro Paraná Norte')
map_sedu = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.Color('TERRITORIO_sedu2017:N', legend=None),tooltip='NM_MUN:N').properties(width=500,height=200, title='Classificação da SEDU')
map_nre = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.Color('TERRITORIO_nre:N', legend=None),tooltip='NM_MUN:N').properties(width=500,height=200, title='Núcleo Regional de Educação')
map_nrs = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.Color('TERRITORIO_nrs:N', legend=None),tooltip='NM_MUN:N').properties(width=500,height=200, title='Núcleo Regional de Saúde')

In [ ]:
first_concat = alt.concat(map_rm, map_arranjo
          ).resolve_scale(
    color='independent'
)

second_concat = alt.concat(map_sedu, map_metronorte
          ).resolve_scale(
    color='independent'
)

third_concat = alt.concat(map_nre, map_nrs
          ).resolve_scale(
    color='independent'
)

## Estrutura Territorial

In [ ]:
chart_estrutura_territorial = alt.vconcat(first_concat, second_concat, third_concat)
chart_estrutura_territorial.save('json_chart_estrutura_territorial.json', format='json')
chart_estrutura_territorial

## Concentração Econômica e Populacional

In [ ]:
interval1 = alt.selection_multi(empty='all') # esta é a seleçção que serve para mapa. Por hora não está em uso.
interval = alt.selection_interval(empty='all', encodings=['x','y']) # esta seleção serve para gráficos

map_area_estudo = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.condition(interval, alt.value('#4682b4'), alt.value('lightgray')), tooltip='NM_MUN:N').properties(width=500,height=200).add_selection(interval1)

base = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.value('lightgray'), tooltip='NM_MUN:N').properties(width=500,height=200)

scatter_pib_pop = alt.Chart(gdf_municipios).mark_point().encode(
    x=alt.X('ESCALA_PIB18_mil:Q', scale=alt.Scale(type='log'), sort='ascending', title='PIB (R$1.000) - Log', axis=alt.Axis(tickCount=10))
    , y=alt.Y('ESCALA_POP_20:Q', sort='ascending', title='População - Log', scale=alt.Scale(type='log'))
#    , color=alt.condition(interval, alt.value('#4682b4'), alt.value('lightgray'))
    , tooltip=['NM_MUN:N', 'ESCALA_PIB18_mil', 'ESCALA_POP_20']
#    alt.condition(interval, 'NM_MUN:N', alt.value('lightgray')
                 
).add_selection(interval).properties(width=500,height=170)

scatter_vaf_ind_pop = alt.Chart(gdf_municipios).mark_point().encode(
    x=alt.X('ESCALA_VAF_18_IND_mil:Q', title='VAF Indústria (R$1000) - Log', scale=alt.Scale(type='log'))
    , y=alt.Y('ESCALA_POP_20:Q', sort='ascending', title='População - Log', scale=alt.Scale(type='log'))
#    , color='TERRITORIO_arranjo:N'
    , tooltip=['NM_MUN:N', 'ESCALA_VAF_18_IND_mil', 'ESCALA_POP_20']
#    alt.condition(interval, 'NM_MUN:N', alt.value('lightgray')
                 
).add_selection(interval).properties(width=500,height=170)

scatter_vaf_serv = alt.Chart(gdf_municipios).mark_point().encode(
    x=alt.X('ESCALA_VAF_18_SERV_mil:Q', scale=alt.Scale(type='log'), sort='ascending', title='VAF Serviços (R$1.000) - Log', axis=alt.Axis(tickCount=10))
    , y=alt.Y('ESCALA_POP_20:Q', sort='ascending', title='População - Log', scale=alt.Scale(type='log'))
#    , color='TERRITORIO_arranjo'
    , tooltip=['NM_MUN:N', 'ESCALA_VAF_18_SERV_mil', 'ESCALA_POP_20']
#    alt.condition(interval, 'NM_MUN:N', alt.value('lightgray')
                 
).add_selection(interval).properties(width=500,height=170)

scatter_vaf_pub_pop = alt.Chart(gdf_municipios).mark_point().encode(
    x=alt.X('ESCALA_VAF_18_PUB_mil:Q', title='VAF Setor Público (R$1000) - Log', scale=alt.Scale(type='log'))
    , y=alt.Y('ESCALA_POP_20:Q', sort='ascending', title='População - Log', scale=alt.Scale(type='log'))
#    ,color='TERRITORIO_arranjo'
    , tooltip=['NM_MUN:N', 'ESCALA_VAF_18_PUB_mil', 'ESCALA_POP_20']
#    alt.condition(interval, 'NM_MUN:N', alt.value('lightgray')
                 
).add_selection(interval).properties(width=500,height=170)


chart_concentracao_econpop =((base + map_area_estudo) & (scatter_pib_pop | scatter_vaf_ind_pop) & (scatter_vaf_serv | scatter_vaf_pub_pop))
chart_concentracao_econpop.save('json_chart_concentracao_econpop.json', format='json')
chart_concentracao_econpop

## Ganhos de Centralidade

In [ ]:
interval1 = alt.selection_multi(empty='all') # esta é a seleçção que serve para mapa. Por hora não está em uso.
interval = alt.selection_interval(empty='all', encodings=['x', 'y']) # esta seleção serve para gráficos

map_area_estudo = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.condition(interval, alt.value('#4682b4'), alt.value('lightgray')), tooltip='NM_MUN:N').properties(width=500,height=200).add_selection(interval1)

base = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.value('lightgray'), tooltip='NM_MUN:N').properties(width=500,height=200)

scatter_pib_pop = alt.Chart(gdf_municipios).mark_point().encode(
    x=alt.X('CENTRALIDADE_PIB_ 07-18', sort='ascending', title='Ganho na Participação do PIB (07-18)', axis=alt.Axis(tickCount=10))
    , y=alt.Y('CENTRALIDADE_vaf_ind_07-20 ', sort='ascending', title='Ganho na Participação da Indústria (07-18)')
#    , color=alt.condition(interval, alt.value('#4682b4'), alt.value('lightgray'))
    , tooltip=['NM_MUN:N', 'CENTRALIDADE_PIB_ 07-18', 'CENTRALIDADE_vaf_ind_07-20 ']
#    alt.condition(interval, 'NM_MUN:N', alt.value('lightgray')
                 
).add_selection(interval).properties(width=500,height=170)

scatter_vaf_ind_pop = alt.Chart(gdf_municipios).mark_point().encode(
    x=alt.X('CENTRALIDADE_vaf_pub_07-20 :Q', title='Ganho na Participação VAF Público (07-18)')
    , y=alt.Y('CENTRALIDADE_vaf_serv_07-20 :Q', sort='ascending', title='Ganho na Participação VAF Serviços (07-20)')
#    , color='TERRITORIO_arranjo:N'
    , tooltip=['NM_MUN:N', 'CENTRALIDADE_vaf_pub_07-20 ', 'CENTRALIDADE_vaf_serv_07-20 ']
#    alt.condition(interval, 'NM_MUN:N', alt.value('lightgray')
                 
).add_selection(interval).properties(width=500,height=170)

scatter_vaf_serv = alt.Chart(gdf_municipios).mark_point().encode(
    x=alt.X('CENTRALIDADE_pop_10-20:Q', sort='ascending', title='Ganho na Participação Populacional (10-20)', axis=alt.Axis(tickCount=10))
    , y=alt.Y('CENTRALIDADE_PIB_ 07-18:Q', sort='ascending', title='Ganho de Participação no PIB (07-18)')
#    , color='TERRITORIO_arranjo'
    , tooltip=['NM_MUN:N', 'CENTRALIDADE_pop_10-20', 'CENTRALIDADE_PIB_ 07-18']
#    alt.condition(interval, 'NM_MUN:N', alt.value('lightgray')
                 
).add_selection(interval).properties(width=500,height=170)


chart_ganhos_centralidade = ((base + map_area_estudo | scatter_vaf_serv) & (scatter_pib_pop | scatter_vaf_ind_pop))
chart_ganhos_centralidade.save('chart_ganhos_centralidade.json', format='json')
chart_ganhos_centralidade

## Graus de Urbanização

In [ ]:
interval1 = alt.selection_multi(empty='all') # esta é a seleçção que serve para mapa. Por hora não está em uso.
interval = alt.selection_interval(empty='all', encodings=['x', 'y']) # esta seleção serve para gráficos


map_area_estudo = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.condition(interval, alt.value('#4682b4'), alt.value('lightgray')), tooltip='NM_MUN:N').properties(width=500,height=200).add_selection(interval1)
base = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.value('lightgray'), tooltip='NM_MUN:N').properties(width=500,height=200)

scatter_pib_pop = alt.Chart(gdf_municipios).mark_point().encode(
    x=alt.X('URBRUR_tipologia:N', sort='ascending', title='Tipologia Urbano-Rural', axis=alt.Axis(tickCount=10))
    , y=alt.Y('URBRUR_tx_urbanizacao_10', sort='ascending', title='Taxa de Urbanização (2010)', scale=alt.Scale(zero=False))
#    , color=alt.condition(interval, alt.value('#4682b4'), alt.value('lightgray'))
    , tooltip=['NM_MUN:N', 'URBRUR_tipologia', 'URBRUR_tx_urbanizacao_10']
#    alt.condition(interval, 'NM_MUN:N', alt.value('lightgray')
                 
).add_selection(interval).properties(width=500,height=170)

scatter_vaf_ind_pop = alt.Chart(gdf_municipios).mark_point().encode(
    x=alt.X('URBRUR_area_urbanizada_embrapa_2015:Q', title='Área Urbanizada', scale=alt.Scale(type='log'))
    , y=alt.Y('URB_RUR_densidade_hab_ha2020:Q', sort='ascending', title='Densidade Demográfica (hab/ha)')
#    , color='TERRITORIO_arranjo:N'
    , tooltip=['NM_MUN:N', 'URBRUR_area_urbanizada_embrapa_2015', 'URB_RUR_densidade_hab_ha2020']
#    alt.condition(interval, 'NM_MUN:N', alt.value('lightgray')
                 
).add_selection(interval).properties(width=500,height=170)

scatter_vaf_serv = alt.Chart(gdf_municipios).mark_point().encode(
    x=alt.X('URB_RUR_hierarquia:N', sort='ascending', title='Hierarquia Urbana', axis=alt.Axis(tickCount=10))
    , y=alt.Y('URB_RUR_densidade_hab_ha2020:Q', sort='ascending', title='Densidade Demográfica (hab/ha)')
#    , color='TERRITORIO_arranjo'
    , tooltip=['NM_MUN:N', 'URB_RUR_hierarquia', 'URB_RUR_densidade_hab_ha2020']
#    alt.condition(interval, 'NM_MUN:N', alt.value('lightgray')
                 
).add_selection(interval).properties(width=500,height=170)


chart_grau_urbanizacao = (base + map_area_estudo | scatter_vaf_serv) & (scatter_pib_pop | scatter_vaf_ind_pop)
chart_grau_urbanizacao.save('chart_grau_urbanizacao.json', format='json')
chart_grau_urbanizacao

## Mobilidade

In [ ]:
interval1 = alt.selection_multi(empty='all') # esta é a seleçção que serve para mapa. Por hora não está em uso.
interval = alt.selection_interval(empty='all', encodings=['y']) # esta seleção serve para gráficos


map_area_estudo = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.condition(interval, alt.value('#4682b4'), alt.value('lightgray')), tooltip='NM_MUN:N').properties(width=500,height=200).add_selection(interval1)
base = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.value('lightgray'), tooltip='NM_MUN:N').properties(width=500,height=400)

scatter_pib_pop = alt.Chart(gdf_municipios).mark_point().encode(
    x=alt.X('MOB_tempo_deslocamento:Q', sort='ascending', title='Tempo de Deslocamento (minutos)', scale=alt.Scale(zero=False))
    ,y=alt.Y('MOB_indice_integracao:Q', title='Índice de Integração', axis=alt.Axis(tickCount=10))
#    , color=alt.condition(interval, alt.value('#4682b4'), alt.value('lightgray'))
    , tooltip=['NM_MUN:N', 'ESCALA_PIB18_mil', 'ESCALA_POP_20']
#    alt.condition(interval, 'NM_MUN:N', alt.value('lightgray')
                 
).add_selection(interval).properties(width=500,height=400)


(base + map_area_estudo | scatter_pib_pop)

In [ ]:
gdf_municipios.columns

In [ ]:
interval1 = alt.selection_multi(empty='all') # esta é a seleçção que serve para mapa. Por hora não está em uso.
interval = alt.selection_interval(empty='all', encodings=['x','y']) # esta seleção serve para gráficos


map_area_estudo = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.condition(interval, alt.value('#4682b4'), alt.value('lightgray')), tooltip='NM_MUN:N').properties(width=500,height=200).add_selection(interval1)
base = alt.Chart(gdf_municipios).mark_geoshape(stroke='black',strokeWidth=1).project().encode(color=alt.value('lightgray'), tooltip='NM_MUN:N').properties(width=500,height=400)

scatter_idh = alt.Chart(gdf_municipios).mark_point().encode(
    x=alt.X('SOCIOE_pib_percapita_2018:Q', sort='ascending', title='PIB per capita', scale=alt.Scale(zero=False))
    ,y=alt.Y('SOCIOE_idh:Q', title='IDH', axis=alt.Axis(tickCount=10), scale=alt.Scale(zero=False))
#    , color=alt.condition(interval, alt.value('#4682b4'), alt.value('lightgray'))
    , tooltip=['NM_MUN:N', 'ESCALA_PIB18_mil', 'ESCALA_POP_20']
#    alt.condition(interval, 'NM_MUN:N', alt.value('lightgray')
                 
).add_selection(interval).properties(width=500,height=400)

scatter_vulnerabilidade = alt.Chart(gdf_municipios).mark_point().encode(
    x=alt.X('SOCIOE_vulnerabilidade:Q', sort='descending', title='Vulnerabilidade Social', scale=alt.Scale(zero=False))
    ,y=alt.Y('SOCIOE_Firjan:Q', title='Índice Firjan', axis=alt.Axis(tickCount=10))
#    , color=alt.condition(interval, alt.value('#4682b4'), alt.value('lightgray'))
    , tooltip=['NM_MUN:N', 'ESCALA_PIB18_mil', 'ESCALA_POP_20']
#    alt.condition(interval, 'NM_MUN:N', alt.value('lightgray')
                 
).add_selection(interval).properties(width=500,height=400)


chart_mobility = ((base + map_area_estudo) & (scatter_idh | scatter_vulnerabilidade))
chart_mobility